In [12]:
import pandas as pd
import numpy as np

# 원본 2021년 데이터 입력 (참조용)
data_2021 = {
    '산업 분류': [
        '취업자 수', '제1차 산업', '농업 임업', '어업', '제2차 산업', '광업 채석업 자갈채취업', '건설업', '제조업',
        '제3차 산업', '전기·가스·열공급·수도업', '정보통신업', '운수업 우편업', '도매업 소매업', '금융업 보험업',
        '부동산업 물품임대업', '학술연구 전문·기술 서비스업', '숙박업 음식 서비스업', '생활관련 서비스업 오락업',
        '교육 학습 지원업', '의료 복지', '복합 서비스 사업', '기타 서비스업 (기타로 분류되지 않은 것)', '공무 (기타로 분류되지 않은 것)',
        '분류 불가능 산업'
    ],
    '레이와 2년 (2021)': [
        2253135, 54590, 58702, 3195, 436067, 661, 178606, 256911, 1687999, 11885, 61097, 131903, 378069,
        53767, 49000, 69144, 124597, 80794, 105381, 321379, 14823, 163158, 79365, 74482
    ]
}

df_2021 = pd.DataFrame(data_2021)

# 새로운 산업 분류 정의
new_categories = {
    '농업, 임업 및 어업 (A)': ['농업 임업', '어업'],
    '광·제조업 (B,C)': ['광업 채석업 자갈채취업', '제조업'],
    '전기·운수·통신·금융 (D,H,J,K)': ['전기·가스·열공급·수도업', '운수업 우편업', '정보통신업', '금융업 보험업'],
    '건설업 (F)': ['건설업'],
    '도소매·숙박음식업 (G,I)': ['도매업 소매업', '숙박업 음식 서비스업'],
    '사업·개인·공공서비스 및 기타 (E,L~U)': [
        '부동산업 물품임대업', '학술연구 전문·기술 서비스업', '생활관련 서비스업 오락업', '교육 학습 지원업',
        '의료 복지', '복합 서비스 사업', '기타 서비스업 (기타로 분류되지 않은 것)', '공무 (기타로 분류되지 않은 것)'
    ],
    '정보통신 (J)': ['정보통신업']
}

# 2021년 데이터를 새로운 산업 분류로 재구성
new_data_2021 = {'산업 분류': list(new_categories.keys()), '레이와 2년 (2021)': []}
for category, old_cats in new_categories.items():
    if category == '정보통신 (J)':
        value = df_2021[df_2021['산업 분류'] == '정보통신업']['레이와 2년 (2021)'].values[0]
    else:
        value = df_2021[df_2021['산업 분류'].isin(old_cats)]['레이와 2년 (2021)'].sum()
    new_data_2021['레이와 2년 (2021)'].append(value)

df_2021_new = pd.DataFrame(new_data_2021)

# 기존 보간된 데이터 입력 (2010~2020년)
data = {
    '산업 분류': [
        '농업, 임업 및 어업 (A)', '광·제조업 (B,C)', '전기·운수·통신·금융 (D,H,J,K)', '건설업 (F)',
        '도소매·숙박음식업 (G,I)', '사업·개인·공공서비스 및 기타 (E,L~U)', '정보통신 (J)'
    ],
    2010: [
        65806, 263891, 298453, 183705, 553104, 857159, 51513
    ],
    2011: [
        64911, 265672, 297295, 182110, 537567, 868361, 52258
    ],
    2012: [
        64015, 267452, 296137, 180514, 522031, 879563, 53003
    ],
    2013: [
        63120, 269233, 294979, 178919, 506494, 890765, 53747
    ],
    2014: [
        62224, 271014, 293821, 177323, 490958, 901967, 54492
    ],
    2015: [
        62642, 276777, 292148, 177709, 502664, 913127, 54772
    ],
    2016: [
        60950, 271068, 294085, 177831, 502665, 924614, 55914
    ],
    2017: [
        59259, 265360, 296022, 177953, 502665, 936101, 57056
    ],
    2018: [
        57567, 259651, 297960, 178074, 502666, 947589, 58198
    ],
    2019: [
        55876, 253943, 299897, 178196, 502666, 959076, 59340
    ],
    2020: [
        54589, 257571, 298650, 178605, 502666, 957398, 61096
    ],
    2021: df_2021_new['레이와 2년 (2021)'].tolist()
}

# 2022년 데이터 선형 보간
data[2022] = []
for i in range(len(data['산업 분류'])):
    y_2020 = data[2020][i]
    y_2021 = data[2021][i]
    # 선형 보간: y_2022 = y_2021 + (y_2021 - y_2020)
    y_2022 = y_2021 + (y_2021 - y_2020)
    data[2022].append(round(y_2022))

# 데이터프레임 생성
df = pd.DataFrame(data)

# 반기별 데이터프레임 생성
np.random.seed(42)  # 재현 가능성을 위해 시드 설정
half_year_data = {'산업 분류': [], '연도': [], '반기': [], '값': []}

# 각 연도와 산업 분류에 대해 반기별 데이터 생성
for year in range(2010, 2023):
    # 50% ± 3% 범위에서 랜덤 상반기 비율 생성 (47%~53%)
    first_half_ratio = np.random.uniform(0.47, 0.53)
    second_half_ratio = 1 - first_half_ratio
    
    for _, row in df.iterrows():
        industry = row['산업 분류']
        yearly_value = row[year]
        
        # 상반기와 하반기 값 계산
        first_half_value = round(yearly_value * first_half_ratio)
        second_half_value = yearly_value - first_half_value  # 나머지
        
        # 상반기 데이터 추가
        half_year_data['산업 분류'].append(industry)
        half_year_data['연도'].append(year)
        half_year_data['반기'].append('상반기')
        half_year_data['값'].append(first_half_value)
        
        # 하반기 데이터 추가
        half_year_data['산업 분류'].append(industry)
        half_year_data['연도'].append(year)
        half_year_data['반기'].append('하반기')
        half_year_data['값'].append(second_half_value)

# 데이터프레임 생성
half_year_df = pd.DataFrame(half_year_data)

# 피벗 테이블로 정리 (연도-반기별로 열 생성)
pivot_df = half_year_df.pivot_table(
    index=['산업 분류'],
    columns=['연도', '반기'],
    values='값',
    aggfunc='sum'
).reset_index()

# 열 이름 정리
pivot_df.columns = ['산업 분류'] + [f'{year}_{half}' for year, half in pivot_df.columns[1:]]

# 행과 열 전치
transposed_df = pivot_df.set_index('산업 분류').transpose().reset_index()
transposed_df = transposed_df.rename(columns={'index': '연도_반기'})

# 결과 출력
print(transposed_df)

# CSV로 저장
transposed_df.to_csv('transposed_half_year_industry_data.csv', index=False)

산업 분류     연도_반기  건설업 (F)  광·제조업 (B,C)  농업, 임업 및 어업 (A)  도소매·숙박음식업 (G,I)  \
0      2010_상반기    90470       129959            32408           272388   
1      2010_하반기    93235       133932            33398           280716   
2      2011_상반기    95980       140021            34211           283321   
3      2011_하반기    86130       125651            30700           254246   
4      2012_상반기    92770       137449            32899           268282   
5      2012_하반기    87744       130003            31116           253749   
6      2013_상반기    90519       136210            31934           256245   
7      2013_하반기    88400       133023            31186           250249   
8      2014_상반기    85002       129914            29828           235346   
9      2014_하반기    92321       141100            32396           255612   
10     2015_상반기    85187       132676            30028           240957   
11     2015_하반기    92522       144101            32614           261707   
12     2016_상반기    84200 